In [3]:
import numpy as np
from numpy.linalg import norm, eigvalsh, eigvals, solve, inv

# Construir matriz y lado derecho
def generarSistema(N):
    A = np.zeros((N, N))
    for i in range(1,N-1):
        A[i, i-1] = -1
        A[i,i] = 2
        A[i,i+1] = -1
    
    A[0,0] = A[N-1, N-1] = 2
    A[0,1] = A[N-1, N-2] = -1
    
    b = np.zeros(N)
    b[:] = 1 # np.ones ...?
    return A, b

A, b = generarSistema(20)

In [4]:
def solveRichardson(A, b, omega, tol, maxit, verbose=False):
    x = b.copy()
    it = 0
    residual = b - A@x
    error = norm(residual)
    if verbose: print(f"It {it:3}, err={error:1.2e}")
    while True:
        x = x + omega * residual
        residual = b - A@x
        error = norm(residual)
        if verbose: print(f"It {it:3}, err={error:1.2e}")
        if error < tol or it > maxit or error > 1e14:
            if error < tol:
                print(f"\tConverged in {it} iterations, error={error}")
            else:
                print("\tDiverged")
            return x
        else:
            it += 1

In [5]:
def solveJacobi(A, b, omega, tol, maxit, verbose=False):
    x = b.copy()
    it = 0
    D = np.diagflat(np.diagonal(A))
    residual = b - A@x
    error = norm(residual)
    if verbose: print(f"It {it:3}, err={error:1.2e}")
    while True:
        dx = solve(D, residual)
        x = x + omega * dx
        residual = b - A@x
        error = norm(residual)
        if verbose: print(f"It {it:3}, err={error:1.2e}")
        if error < tol or it > maxit or error > 1e14:
            if error < tol:
                print(f"\tConverged in {it} iterations, error={error}")
            else:
                print("\tDiverged")
            return x
        else:
            it += 1

In [6]:
def solveGaussSeidel(A, b, omega, tol, maxit, verbose=False):
    x = b.copy()
    it = 0
    L = np.tril(A)
    residual = b - A@x
    error = norm(residual)
    if verbose: print(f"It {it:3}, err={error:1.2e}")
    while True:
        dx = solve(L, residual)
        x = x + omega * dx
        residual = b - A@x
        error = norm(residual)
        if verbose: print(f"It {it:3}, err={error:1.2e}")
        if error < tol or it > maxit or error > 1e14:
            if error < tol:
                print(f"\tConverged in {it} iterations, error={error}")
            else:
                print("\tDiverged")
            return x
        else:
            it += 1

In [8]:
tol = 1e-6
maxit = 10000

# Richardson tests
eigs = eigvalsh(A)
omega = 1
xsol = solveRichardson(A, b, omega, tol, maxit)
omega = 2/(max(eigs) + min(eigs))
xsol = solveRichardson(A, b, omega, tol, maxit)

	Diverged
	Converged in 1354 iterations, error=9.889812122702099e-07


In [15]:
# Jacobi tests
omega = 1
xsol = solveJacobi(A, b, omega, tol, maxit)
D = np.diagflat(np.diagonal(A))
Dinv = inv(D)
DinvA = Dinv @ A
eigs = eigvals(DinvA)
omega = 2/(max(eigs) + min(eigs))
xsol = solveJacobi(A, b, omega, tol, maxit)

	Converged in 1354 iterations, error=9.889812122702099e-07
	Converged in 1354 iterations, error=9.889812122702099e-07


In [16]:
# Gauss-Seidel
omega = 1
xsol = solveGaussSeidel(A, b, omega, tol, maxit)
L = np.tril(A)
Linv = inv(L)
LinvA = Linv @ A
eigs = eigvals(LinvA)
omega = 2/(max(eigs) + min(eigs))
_ = solveGaussSeidel(A, b, omega, tol, maxit) # Ver que omega tiene vals complejos

	Converged in 678 iterations, error=9.79615736689055e-07
	Diverged


In [14]:
A @ xsol - b

array([-9.79859465e-08, -1.42625680e-07, -1.83105577e-07, -2.18618837e-07,
       -2.48486899e-07, -2.72171690e-07, -2.89284486e-07, -2.99591065e-07,
       -3.03013692e-07, -2.99629278e-07, -2.89664214e-07, -2.73486009e-07,
       -2.51592084e-07, -2.24595709e-07, -1.93209985e-07, -1.58229511e-07,
       -1.20511274e-07, -8.09538960e-08, -4.04769480e-08,  0.00000000e+00])